# Test covariance matrix setup

In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import sys
sys.path.insert(0, "../src")

In [42]:
import numpy as np
import pandas as pd
import xarray
import gstools

import geostats as gs

In [28]:
# read 5-degree monthly data and subset to US for faster testing
ds = xarray.open_dataset("../data/exp_pro/OCO2_5deg_monthly.nc")

ds = ds.where((-125 < ds.lon) & (ds.lon < -70) 
              & (25 < ds.lat) & (ds.lat < 50), drop=True)

ds

<xarray.Dataset>
Dimensions:  (lat: 5, lon: 11, time: 71)
Coordinates:
  * lon      (lon) float64 -122.5 -117.5 -112.5 -107.5 ... -82.5 -77.5 -72.5
  * lat      (lat) float64 27.5 32.5 37.5 42.5 47.5
  * time     (time) datetime64[ns] 2014-09-01 2014-10-01 ... 2020-07-01
Data variables:
    sif      (lon, lat, time) float32 nan nan nan ... 0.49874693 0.60988253
    xco2     (lon, lat, time) float32 nan nan nan ... 412.90573 409.11713

In [29]:
Z1 = gs.Field(DS_grid.xco2, "2019-07-01")
Z2 = gs.Field(DS_grid.sif, "2019-07-01")

In [32]:
model = gstools.Matern(dim=2, nu=2.5)
krige = gs.Cokrige(Z1, Z2, model, fast_dist=True)

In [34]:
dists = krige._get_dists()

In [36]:
dists["block_11"].shape

(2520, 2520)

In [38]:
model.cov_nugget(dists["block_12"])

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [43]:
cov_mat = krige._get_cov_mat()

In [45]:
cov_mat.shape

(5040, 5040)